# Test Modules

In [26]:
# Enable autoreload for development
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# Set project root and add src to path
import sys
from pathlib import Path
import os

PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
project_root = Path(PROJECT_ROOT)
src_path = project_root / 'src'

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

In [28]:
# Print resource summary
from word2gm_fast.utils.resource_summary import print_resource_summary

print_resource_summary()

<pre>SYSTEM RESOURCE SUMMARY
=============================================
Hostname: cm045.hpc.nyu.edu

Job Allocation:
   CPUs: 14
   Memory: 125.0 GB
   Partition: short
   Job ID: 63785051
   Node list: cm045

Physical GPU Hardware:
   No physical GPUs allocated to this job

TensorFlow GPU Recognition:
   TensorFlow can access 0 GPU(s)
   Built with CUDA support: True
=============================================</pre>

## Test `corpus_to_dataset.py`

In [29]:
import subprocess

# Run the corpus_to_dataset test
result = subprocess.run([
    'python', '-m', 'pytest', 
    'tests/test_corpus_to_dataset.py',
    '-v'
], capture_output=True, text=True, cwd=PROJECT_ROOT)

print(result.stdout)
if result.stderr:
    print(result.stderr)

============================= test session starts ==============================
platform linux -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0 -- /ext3/miniforge3/envs/word2gm-fast2/bin/python
cachedir: .pytest_cache
rootdir: /scratch/edk202/word2gm-fast
plugins: anyio-4.9.0, timeout-2.4.0
collecting ... collected 12 items

tests/test_corpus_to_dataset.py::TestValidate5gramLine::test_valid_cases PASSED [  8%]
tests/test_corpus_to_dataset.py::TestValidate5gramLine::test_invalid_cases PASSED [ 16%]
tests/test_corpus_to_dataset.py::TestValidate5gramLine::test_validation_rules PASSED [ 25%]
tests/test_corpus_to_dataset.py::TestMakeDataset::test_filtering_accuracy PASSED [ 33%]
tests/test_corpus_to_dataset.py::TestMakeDataset::test_summary_statistics PASSED [ 41%]
tests/test_corpus_to_dataset.py::TestMakeDataset::test_caching_functionality PASSED [ 50%]
tests/test_corpus_to_dataset.py::TestMakeDataset::test_preview_integration PASSED [ 58%]
tests/test_corpus_to_dataset.py::TestMakeDataset::t

## Test `dataset_to_frequency.py`

In [30]:
import subprocess

# Run the corpus_to_dataset test
result = subprocess.run([
    'python', '-m', 'pytest', 
    'tests/test_dataset_to_frequency.py',
    '-v'
], capture_output=True, text=True, cwd=PROJECT_ROOT)

print(result.stdout)
if result.stderr:
    print(result.stderr)

============================= test session starts ==============================
platform linux -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0 -- /ext3/miniforge3/envs/word2gm-fast2/bin/python
cachedir: .pytest_cache
rootdir: /scratch/edk202/word2gm-fast
plugins: anyio-4.9.0, timeout-2.4.0
collecting ... collected 11 items

tests/test_dataset_to_frequency.py::TestDatasetToFrequency::test_basic_frequency_counting PASSED [  9%]
tests/test_dataset_to_frequency.py::TestDatasetToFrequency::test_frequency_counts PASSED [ 18%]
tests/test_dataset_to_frequency.py::TestDatasetToFrequency::test_sorted_by_frequency PASSED [ 27%]
tests/test_dataset_to_frequency.py::TestDatasetToFrequency::test_empty_dataset PASSED [ 36%]
tests/test_dataset_to_frequency.py::TestDatasetToFrequency::test_single_line_dataset PASSED [ 45%]
tests/test_dataset_to_frequency.py::TestDatasetToFrequency::test_repeated_tokens_same_line PASSED [ 54%]
tests/test_dataset_to_frequency.py::TestDatasetToFrequency::test_whitespace_han

## Test `dataset_to_triplets.py`

In [31]:
import subprocess

# Run the corpus_to_dataset test
result = subprocess.run([
    'python', '-m', 'pytest', 
    'tests/test_dataset_to_triplets.py',
    '-v'
], capture_output=True, text=True, cwd=PROJECT_ROOT)

print(result.stdout)
if result.stderr:
    print(result.stderr)

============================= test session starts ==============================
platform linux -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0 -- /ext3/miniforge3/envs/word2gm-fast2/bin/python
cachedir: .pytest_cache
rootdir: /scratch/edk202/word2gm-fast
plugins: anyio-4.9.0, timeout-2.4.0
collecting ... collected 10 items

tests/test_dataset_to_triplets.py::TestDatasetToTriplets::test_basic_triplet_generation PASSED [ 10%]
tests/test_dataset_to_triplets.py::TestDatasetToTriplets::test_triplet_structure PASSED [ 20%]
tests/test_dataset_to_triplets.py::TestDatasetToTriplets::test_unk_exclusion PASSED [ 30%]
tests/test_dataset_to_triplets.py::TestDatasetToTriplets::test_empty_dataset PASSED [ 40%]
tests/test_dataset_to_triplets.py::TestDatasetToTriplets::test_caching_functionality PASSED [ 50%]
tests/test_dataset_to_triplets.py::TestDatasetToTriplets::test_summary_statistics PASSED [ 60%]
tests/test_dataset_to_triplets.py::TestEdgeCases::test_single_line_dataset PASSED [ 70%]
tests/test_d

## Test `index_vocab.py`

In [32]:
import subprocess

# Run the index_vocab test
result = subprocess.run([
    'python', '-m', 'pytest', 
    'tests/test_index_vocab.py',
    '-v'
], capture_output=True, text=True, cwd=PROJECT_ROOT)

print(result.stdout)
if result.stderr:
    print(result.stderr)

============================= test session starts ==============================
platform linux -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0 -- /ext3/miniforge3/envs/word2gm-fast2/bin/python
cachedir: .pytest_cache
rootdir: /scratch/edk202/word2gm-fast
plugins: anyio-4.9.0, timeout-2.4.0
collecting ... collected 18 items

tests/test_index_vocab.py::TestBuildVocabTable::test_basic_vocab_table_creation PASSED [  5%]
tests/test_index_vocab.py::TestBuildVocabTable::test_unk_token_validation PASSED [ 11%]
tests/test_index_vocab.py::TestBuildVocabTable::test_lookup_unknown_tokens PASSED [ 16%]
tests/test_index_vocab.py::TestMakeVocab::test_basic_vocab_creation PASSED [ 22%]
tests/test_index_vocab.py::TestMakeVocab::test_vocab_ordering PASSED     [ 27%]
tests/test_index_vocab.py::TestMakeVocab::test_frequency_counting PASSED [ 33%]
tests/test_index_vocab.py::TestMakeVocab::test_empty_dataset PASSED      [ 38%]
tests/test_index_vocab.py::TestTripletsToIntegers::test_basic_conversion PASSED [ 

## Test `io.triplets.py`

In [33]:
import subprocess

# Run the index_vocab test
result = subprocess.run([
    'python', '-m', 'pytest', 
    'tests/test_io_triplets.py',
    '-v'
], capture_output=True, text=True, cwd=PROJECT_ROOT)

print(result.stdout)
if result.stderr:
    print(result.stderr)

============================= test session starts ==============================
platform linux -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0 -- /ext3/miniforge3/envs/word2gm-fast2/bin/python
cachedir: .pytest_cache
rootdir: /scratch/edk202/word2gm-fast
plugins: anyio-4.9.0, timeout-2.4.0
collecting ... collected 13 items

tests/test_io_triplets.py::TestWriteTripletsToTFRecord::test_basic_write_uncompressed PASSED [  7%]
tests/test_io_triplets.py::TestWriteTripletsToTFRecord::test_basic_write_compressed PASSED [ 15%]
tests/test_io_triplets.py::TestWriteTripletsToTFRecord::test_empty_dataset PASSED [ 23%]
tests/test_io_triplets.py::TestWriteTripletsToTFRecord::test_large_integers PASSED [ 30%]
tests/test_io_triplets.py::TestLoadTripletsFromTFRecord::test_basic_load_uncompressed PASSED [ 38%]
tests/test_io_triplets.py::TestLoadTripletsFromTFRecord::test_basic_load_compressed PASSED [ 46%]
tests/test_io_triplets.py::TestLoadTripletsFromTFRecord::test_load_empty_file PASSED [ 53%]
tests/te

## Test `io.vocab.py`

In [34]:
import subprocess

# Run the io.vocab test
result = subprocess.run([
    'python', '-m', 'pytest', 
    'tests/test_io_vocab.py',
    '-v'
], capture_output=True, text=True, cwd=PROJECT_ROOT)

print(result.stdout)
if result.stderr:
    print(result.stderr)

============================= test session starts ==============================
platform linux -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0 -- /ext3/miniforge3/envs/word2gm-fast2/bin/python
cachedir: .pytest_cache
rootdir: /scratch/edk202/word2gm-fast
plugins: anyio-4.9.0, timeout-2.4.0
collecting ... collected 16 items

tests/test_io_vocab.py::TestWriteVocabToTFRecord::test_basic_write_uncompressed PASSED [  6%]
tests/test_io_vocab.py::TestWriteVocabToTFRecord::test_basic_write_compressed PASSED [ 12%]
tests/test_io_vocab.py::TestWriteVocabToTFRecord::test_write_without_frequencies PASSED [ 18%]
tests/test_io_vocab.py::TestWriteVocabToTFRecord::test_empty_vocab PASSED [ 25%]
tests/test_io_vocab.py::TestLoadVocabFromTFRecord::test_basic_load_uncompressed PASSED [ 31%]
tests/test_io_vocab.py::TestLoadVocabFromTFRecord::test_basic_load_compressed PASSED [ 37%]
tests/test_io_vocab.py::TestLoadVocabFromTFRecord::test_load_without_frequencies PASSED [ 43%]
tests/test_io_vocab.py::TestLoad

## Test `io.artifacts.py`

In [35]:
import subprocess

# Run the io.artifacts test
result = subprocess.run([
    'python', '-m', 'pytest', 
    'tests/test_io_artifacts.py',
    '-v'
], capture_output=True, text=True, cwd=PROJECT_ROOT)

print(result.stdout)
if result.stderr:
    print(result.stderr)

============================= test session starts ==============================
platform linux -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0 -- /ext3/miniforge3/envs/word2gm-fast2/bin/python
cachedir: .pytest_cache
rootdir: /scratch/edk202/word2gm-fast
plugins: anyio-4.9.0, timeout-2.4.0
collecting ... collected 13 items

tests/test_io_artifacts.py::TestSavePipelineArtifacts::test_basic_save_uncompressed PASSED [  7%]
tests/test_io_artifacts.py::TestSavePipelineArtifacts::test_basic_save_compressed PASSED [ 15%]
tests/test_io_artifacts.py::TestSavePipelineArtifacts::test_directory_creation PASSED [ 23%]
tests/test_io_artifacts.py::TestSavePipelineArtifacts::test_empty_triplets PASSED [ 30%]
tests/test_io_artifacts.py::TestLoadPipelineArtifacts::test_basic_load_uncompressed PASSED [ 38%]
tests/test_io_artifacts.py::TestLoadPipelineArtifacts::test_basic_load_compressed PASSED [ 46%]
tests/test_io_artifacts.py::TestLoadPipelineArtifacts::test_auto_detect_compression PASSED [ 53%]
tests/t

## Test `io.tables.py`

In [36]:
import subprocess

# Run the io.tables test
result = subprocess.run([
    'python', '-m', 'pytest', 
    'tests/test_io_tables.py',
    '-v'
], capture_output=True, text=True, cwd=PROJECT_ROOT)

print(result.stdout)
if result.stderr:
    print(result.stderr)

============================= test session starts ==============================
platform linux -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0 -- /ext3/miniforge3/envs/word2gm-fast2/bin/python
cachedir: .pytest_cache
rootdir: /scratch/edk202/word2gm-fast
plugins: anyio-4.9.0, timeout-2.4.0
collecting ... collected 17 items

tests/test_io_tables.py::TestCreateTokenToIndexTable::test_basic_creation_uncompressed PASSED [  5%]
tests/test_io_tables.py::TestCreateTokenToIndexTable::test_basic_creation_compressed PASSED [ 11%]
tests/test_io_tables.py::TestCreateTokenToIndexTable::test_with_triplet_filtering PASSED [ 17%]
tests/test_io_tables.py::TestCreateTokenToIndexTable::test_table_size PASSED [ 23%]
tests/test_io_tables.py::TestCreateTokenToIndexTable::test_nonexistent_file PASSED [ 29%]
tests/test_io_tables.py::TestCreateIndexToTokenTable::test_basic_creation_uncompressed PASSED [ 35%]
tests/test_io_tables.py::TestCreateIndexToTokenTable::test_basic_creation_compressed PASSED [ 41%]
tests

## Test `pipeline.py`

In [43]:
import subprocess

# Run the io.tables test
result = subprocess.run([
    'python', '-m', 'pytest', 
    'tests/test_pipeline.py',
    '-v'
], capture_output=True, text=True, cwd=PROJECT_ROOT)

print(result.stdout)
if result.stderr:
    print(result.stderr)

============================= test session starts ==============================
platform linux -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0 -- /ext3/miniforge3/envs/word2gm-fast2/bin/python
cachedir: .pytest_cache
rootdir: /scratch/edk202/word2gm-fast
plugins: anyio-4.9.0, timeout-2.4.0
collecting ... collected 26 items

tests/test_pipeline.py::TestProcessSingleYear::test_successful_processing PASSED [  3%]
tests/test_pipeline.py::TestProcessSingleYear::test_compressed_output PASSED [  7%]
tests/test_pipeline.py::TestProcessSingleYear::test_nonexistent_file PASSED [ 11%]
tests/test_pipeline.py::TestProcessSingleYear::test_invalid_corpus_directory PASSED [ 15%]
tests/test_pipeline.py::TestProcessSingleYear::test_small_corpus_file PASSED [ 19%]
tests/test_pipeline.py::TestProcessYearRange::test_single_year_range PASSED [ 23%]
tests/test_pipeline.py::TestProcessYearRange::test_multiple_years_sequential PASSED [ 26%]
tests/test_pipeline.py::TestProcessYearRange::test_year_range_dash_nota